In [12]:
!pip install pyspark
!pip install findspark

In [20]:
import pyspark
import findspark
import wget
from pyspark.sql import SparkSession

In [21]:
wget.download('https://www.dropbox.com/s/meprg01g3nsxo9b/2015-summary.csv?dl=1')

100% [................................................................................] 7080 / 7080

'2015-summary (1).csv'

In [25]:
spark = SparkSession.builder.master("local")\
.appName("Word Count")\
.config("spark.some.config.option", "some-value")\
.getOrCreate()

In [26]:
myRange = spark.range(1000).toDF('number')

In [32]:
myRange

DataFrame[number: bigint]

In [31]:
divisBy2 = myRange.where('number % 2 = 0')

In [33]:
divisBy2

DataFrame[number: bigint]

In [34]:
divisBy2.count()

500

In [35]:
flightData2015 = spark.read.option('inferSchema','true').option('header','true').csv('2015-summary.csv')

In [37]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [39]:
flightData2015.sort('count').explain()

== Physical Plan ==
*(2) Sort [count#27 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#27 ASC NULLS FIRST, 200)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#25,ORIGIN_COUNTRY_NAME#26,count#27] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/D:/Dropbox/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


In [40]:
spark.conf.set('spark.sql.shuffle.partitions','5')

In [41]:
flightData2015.sort('count').take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]